In [3]:
#import library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool, cv

from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
import pickle

pd.set_option('display.max_columns', None)
%matplotlib inline

In [4]:
df = pd.read_csv("/home/chewye/Документы/data/dataset.csv")

/tmp/ipykernel_31449/2469748590.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/chewye/Документы/data/dataset.csv")


## EDA

In [5]:
df.head()

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629.0,Участок,7503.0,N,0.0,Y,N,87.0,5.0,1.0,0.0,2.0,0.00,0.0,41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976.0,ММПО,7503.0,N,0.0,N,N,107.0,5.0,1.0,0.0,2.0,0.00,0.0,68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962.0,Цех,7503.0,N,0.0,Y,N,50.0,5.0,1.0,0.0,2.0,0.00,0.0,56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0
3,9020937,1019_-1,344964.0,Цех,7503.0,N,0.0,Y,N,416.0,5.0,1.0,0.0,2.0,35.34,0.0,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0
4,3082311,1020_-1,629819.0,Участок,7503.0,N,0.0,Y,N,795.0,5.0,1.0,0.0,2.0,52.52,0.0,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0


In [6]:
#Check null values
df.isnull().sum()

id                               0
oper_type + oper_attr            0
index_oper                       0
type                             0
priority                         0
is_privatecategory               0
class                            0
is_in_yandex                     0
is_return                        0
weight                           0
mailtype                         0
mailctg                          0
mailrank                         0
directctg                        0
transport_pay                    0
postmark                         0
weight_mfi                       0
price_mfi                        0
dist_qty_oper_login_1            0
total_qty_oper_login_1           0
total_qty_oper_login_0           0
total_qty_over_index_and_type    0
total_qty_over_index             0
is_wrong_sndr_name               0
is_wrong_rcpn_name               0
is_wrong_phone_number            0
is_wrong_address                 0
label                            0
dtype: int64

In [7]:
#Check features about type  
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000000 entries, 0 to 5999999
Data columns (total 28 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   id                             int64  
 1   oper_type + oper_attr          object 
 2   index_oper                     object 
 3   type                           object 
 4   priority                       float64
 5   is_privatecategory             object 
 6   class                          float64
 7   is_in_yandex                   object 
 8   is_return                      object 
 9   weight                         float64
 10  mailtype                       float64
 11  mailctg                        float64
 12  mailrank                       float64
 13  directctg                      float64
 14  transport_pay                  float64
 15  postmark                       float64
 16  weight_mfi                     float64
 17  price_mfi                      float64
 18  di

In [8]:
df.describe()

,id,priority,class,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
count,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6000000.0,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06
mean,4.999167e+06,7.499824e+03,3.571923e-01,2.399936e+02,4.995037e+00,7.726733e-01,0.0,1.988388e+00,1.187661e+01,9.069800e-02,1.236553e+02,4.560945e+02,3.190689e+02,2.775095e+07,2.898887e+07,5.673982e+07,7.178686e+07,2.564167e-03,2.597737e-01,1.709710e-01,1.262833e-03,2.849667e-02
std,2.886491e+06,1.751917e+02,9.861573e-01,4.504002e+02,1.562659e-01,4.407572e-01,0.0,1.073716e-01,2.224794e+01,7.296577e+00,2.084230e+02,2.103291e+03,4.012494e+02,2.903655e+07,5.334262e+07,7.290388e+07,7.509270e+07,5.057264e-02,4.385104e-01,3.764837e-01,3.551392e-02,1.663869e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.500126e+06,7.503000e+03,0.000000e+00,5.200000e+01,5.000000e+00,1.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,2.300000e+01,5.000000e+01,3.100000e+01,1.113900e+06,1.064800e+04,1.673370e+06,1.040204e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.999348e+06,7.503000e+03,0.000000e+00,1.050000e+02,5.000000e+00,1.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,6.000000e+01,1.920000e+02,1.130000e+02,9.570832e+06,3.664710e+05,1.393496e+07,5.270807e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,7.498348e+06,7.506000e+03,0.000000e+00,2.640000e+02,5.000000e+00,1.000000e+00,0.0,2.000000e+00,2.295000e+01,0.000000e+00,1.230000e+02,4.180000e+02,9.140000e+02,5.610820e+07,8.331893e+07,1.321756e+08,1.368198e+08,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.999999e+06,7.506000e+03,5.000000e+00,1.580000e+05,9.000000e+00,5.000000e+00,0.0,2.000000e+00,8.305000e+02,1.024000e+03,1.800000e+04,1.597050e+05,1.142000e+03,1.416888e+08,4.156581e+08,4.897385e+08,5.002511e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [9]:
numerical_col = [
    'weight',
    'weight_mfi',
    'price_mfi', 
    'transport_pay',
    'dist_qty_oper_login_1',
    'total_qty_oper_login_1',
    'total_qty_oper_login_0',
    'total_qty_over_index_and_type',
    'total_qty_over_index',
    'label'
]

In [10]:
#View how much values = 0 in numerical features
for feature in numerical_col:
  print(f'{feature}: {df[df[feature]==0][feature].count()}')

weight: 4893
weight_mfi: 816226
price_mfi: 816663
transport_pay: 4120507
dist_qty_oper_login_1: 16708
total_qty_oper_login_1: 16708
total_qty_oper_login_0: 6881
total_qty_over_index_and_type: 5670
total_qty_over_index: 5670
label: 5829020


In [11]:
df.select_dtypes(include='object')

,oper_type + oper_attr,index_oper,type,is_privatecategory,is_in_yandex,is_return
0,1043_-1,628629.0,Участок,N,Y,N
1,1023_-1,102976.0,ММПО,N,N,N
2,1018_-1,620962.0,Цех,N,Y,N
3,1019_-1,344964.0,Цех,N,Y,N
4,1020_-1,629819.0,Участок,N,Y,N
...,...,...,...,...,...,...
5999995,1022_-1,102976.0,ММПО,N,N,N
5999996,1022_-1,241963.0,Цех,N,Y,N
5999997,1041_-1,102971.0,Цех,N,N,N
5999998,1018_-1,102152.0,Цех,N,N,N


In [12]:
#Check how much empty strings in features type object
for col in ['oper_type + oper_attr', 'index_oper', 'type', 'is_privatecategory', 'is_in_yandex', 'is_return']:
    print(f'{col}: {df[df[col]==" "].shape[0]}')

oper_type + oper_attr: 0
index_oper: 154
type: 0
is_privatecategory: 0
is_in_yandex: 0
is_return: 0


In [13]:
#Change empty values in "index_oper" on 0
df.loc[(df['index_oper'] == ' '), 'index_oper'] = '0' 
df['index_oper'] = df['index_oper'].astype(np.float64)

In [14]:
#Split index_oper on two features
df['oper_type'] = df['oper_type + oper_attr'].str.partition('_')[0]
df['oper_attr'] = df['oper_type + oper_attr'].str.partition('_')[2]

df.head()

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label,oper_type,oper_attr
0,6818780,1043_-1,628629.0,Участок,7503.0,N,0.0,Y,N,87.0,5.0,1.0,0.0,2.0,0.00,0.0,41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0,1043,-1
1,9907176,1023_-1,102976.0,ММПО,7503.0,N,0.0,N,N,107.0,5.0,1.0,0.0,2.0,0.00,0.0,68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0,1023,-1
2,3304275,1018_-1,620962.0,Цех,7503.0,N,0.0,Y,N,50.0,5.0,1.0,0.0,2.0,0.00,0.0,56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0,1018,-1
3,9020937,1019_-1,344964.0,Цех,7503.0,N,0.0,Y,N,416.0,5.0,1.0,0.0,2.0,35.34,0.0,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0,1019,-1
4,3082311,1020_-1,629819.0,Участок,7503.0,N,0.0,Y,N,795.0,5.0,1.0,0.0,2.0,52.52,0.0,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0,1020,-1


In [15]:
# Index_oper - code deparment, reduce him to 3 sign, which show town department
df['index_oper'] = df['index_oper'] // 1000
df['index_oper'] 

0          628.0
1          102.0
2          620.0
3          344.0
4          629.0
           ...  
5999995    102.0
5999996    241.0
5999997    102.0
5999998    102.0
5999999    140.0
Name: index_oper, Length: 6000000, dtype: float64

In [16]:
# Change categorical features type float -> int
int_col = [
    'index_oper',
    'priority',
    'class',
    'mailtype', 
    'mailctg', 
    'directctg', 
    'postmark',
]

df[int_col] = df[int_col].astype(np.int64)

In [17]:
del_col = ['mailrank', 'id', 'oper_type + oper_attr', 'transport_pay', 'weight_mfi', 'transport_pay']

categorical_col = [
 'index_oper',
 'type',
 'priority', 
 'class',
 'is_privatecategory', 
 'is_in_yandex', 
 'is_return', 
 'mailtype', 
 'mailctg', 
 'directctg', 
 'postmark',
 'is_wrong_sndr_name',
 'is_wrong_rcpn_name',
 'is_wrong_phone_number',
 'is_wrong_address',
 'oper_type',
 'oper_attr'
 ]

In [18]:
df.drop(columns=del_col, axis=1, inplace=True)

In [19]:
df.head()

,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,directctg,postmark,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label,oper_type,oper_attr
0,628,Участок,7503,N,0,Y,N,87.0,5,1,2,0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0,1043,-1
1,102,ММПО,7503,N,0,N,N,107.0,5,1,2,0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0,1023,-1
2,620,Цех,7503,N,0,Y,N,50.0,5,1,2,0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0,1018,-1
3,344,Цех,7503,N,0,Y,N,416.0,5,1,2,0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0,1019,-1
4,629,Участок,7503,N,0,Y,N,795.0,5,1,2,0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0,1020,-1


In [21]:
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

x_train = train.drop(['label'], axis=1)
y_train = train['label']

x_test = test.drop(['label'], axis=1)
y_test = test['label']

## Feature Engineering

We pick important features use catboost

In [22]:
train_data = Pool(
            x_train,
            label=y_train,
            cat_features=categorical_col,      
            )

In [23]:
params = {'iterations': 200,
          'learning_rate': 0.05,
          'verbose': False,
          'loss_function': 'Logloss',
          'eval_metric': 'AUC',
          'random_seed': 42
          }

In [24]:
score = CatBoostClassifier(**params)
score.fit(train_data, plot=True, eval_set=(x_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [26]:
score.get_feature_importance(prettified=True)

,Feature Id,Importances
0,oper_type,28.216761
1,total_qty_oper_login_1,20.436031
2,dist_qty_oper_login_1,15.962054
3,mailctg,7.740863
4,total_qty_oper_login_0,5.646996
5,price_mfi,4.222162
6,weight,3.947483
7,total_qty_over_index_and_type,3.309264
8,total_qty_over_index,2.227766
9,directctg,2.153605


In [27]:
list_del = list(score.get_feature_importance(prettified=True)['Feature Id'])[13:]

In [28]:
categorical_col = [
 'type',
 'mailctg', 
 'directctg', 
 'is_wrong_rcpn_name',
 'is_wrong_phone_number',
 'oper_type'
  ]
df.drop(columns=list_del ,axis=1, inplace=True)

## Model_1 Catboost

In [29]:
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

x_train = train.drop(['label'], axis=1)
y_train = train['label']

x_test = test.drop(['label'], axis=1)
y_test = test['label']

In [30]:
train_data = Pool(
            x_train,
            label=y_train,
            cat_features=categorical_col,      
            )

In [31]:
model = CatBoostClassifier(
    iterations=200,
    random_seed=42,
    learning_rate=0.04,
    eval_metric='AUC',
    l2_leaf_reg=3,
    depth=10
)

In [32]:
model.fit(
    train_data,
    eval_set=(x_test, y_test),
    verbose=False,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [33]:
y_preds = model.predict_proba(x_test)[:,1]

In [34]:
from sklearn.metrics import roc_curve

In [35]:
fpr, tpr, thresholds = roc_curve(y_test, y_preds)

In [36]:
data = {'fpr':fpr, 'tpr': tpr, 'thresholds': thresholds}
roc = pd.DataFrame(data)
roc['roc_auc'] = (1 + roc['tpr'] - roc['fpr']) / 2
roc.head()

,fpr,tpr,thresholds,roc_auc
0,0.000000e+00,0.000000,1.983568,0.500000
1,0.000000e+00,0.000029,0.983568,0.500015
2,0.000000e+00,0.006258,0.903916,0.503129
3,8.577771e-07,0.006258,0.903867,0.503129
4,8.577771e-07,0.006580,0.873886,0.503289


In [37]:
roc[roc['roc_auc']==roc['roc_auc'].max()]['thresholds']

26928    0.057939
Name: thresholds, dtype: float64

In [38]:
top_thr = top_thresholds_catboost = roc[roc['roc_auc']==roc['roc_auc'].max()].thresholds

## Model 2

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [40]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[categorical_col])

OneHotEncoder(handle_unknown='ignore')

In [41]:
df_ = enc.transform(df[categorical_col]).toarray()

In [42]:
data = pd.DataFrame(data=df_)

In [43]:
df.drop(columns=categorical_col, inplace=True)

In [44]:
df = pd.concat([df, data], sort=False, axis=1)

In [45]:
df = df.astype(np.int64)

In [46]:
df.columns = list(map(str, list(df.columns)))

In [47]:
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

x_train = train.drop(['label'], axis=1)
y_train = train['label']

x_test = test.drop(['label'], axis=1)
y_test = test['label']

In [48]:
model_log = LogisticRegression().fit(x_train, y_train)

In [49]:
from sklearn.metrics import roc_auc_score

In [50]:
pred_log = model_log.predict_proba(x_test)[:,1]


In [51]:
roc_auc_score(y_test, pred_log)

0.684140107406314

## Final

So, we build model gradient boosting on decision trees by using catboost, which easy work with categorical featuers. Second model classic logistic regression by using sklearn. Catboost become very much better.

In [52]:
output_file = 'model.bin'

In [53]:
with open(output_file, 'wb') as f_out:
    pickle.dump(model, f_out)

In [54]:
top_thr_output = 'thr.bin'


In [55]:
with open(top_thr_output, 'wb') as f_out:
    pickle.dump(float(top_thr), f_out)